# Аппаратное ускорение вычислений
## Клюкин Валерий
## БПИ173

# Мотивация
---
Пример:
<center><img src="pictures/motivation1.png" width="400" align="center"/><center/>

<center><img src="pictures/motivation2.png" width="400" align="center"/><center/>

<center><b>
Вывод: скорость произведения матриц важна для оптимизации работы нейросетей
</b></center>

# Матричное умножение
---
**Определение**: Произведением матриц $А$ размера $n \times k$ и $B$ размера $k \times m$ является матрица $C$ размера $n \times m$ с элементами, равными:
$$c_{ij} = \sum \limits_{s = 1}^{k}a_{is}b_{sj}, \\
i = 1, \dots, n; j=1, \dots, m$$

При размерности $m=k=n$ наивный алгоритм осуществляет $2n^3 - n^2 = \mathcal{O}(n^3)$ операций.

# Матричное умножение
---
* Лежит в основе многих эффективных алгоритмов линейной алгебры. (*QR, LU, SVD - разложения*, *итеративные методы решения CЛАУ*)
* Inference нейронной сети тоже представляет собой произведение матриц

# Почему наивный алгоритм медленный?
---
1. Не использует преимущества архитектуры памяти компьютера.
2. Не использует возможность параллелизации вычислений.

# Иерархия памяти
---
<center><img src="pictures/Memory-Hierarchy.jpg" width="700" align="center"/><center/>

* Быстрая память - дорогая
* Медленной памяти гораздо больше

**Идея:**
Разбить матрицы на блоки. Для примера рассмотрим блочные матрицы $2 \times 2$:
$$A = \begin{bmatrix}
  A_{11} & A_{12}\\
  A_{21} & A_{22}\\
\end{bmatrix}, B = \begin{bmatrix}
  B_{11} & B_{12}\\
  B_{21} & B_{22}\\
\end{bmatrix}$$
Тогда:
$$A \cdot B = \begin{bmatrix}
  A_{11} B_{11} + A_{12} B_{21} & A_{11} B_{12} + A_{12} B_{22}\\
  A_{21} B_{11} + A_{22} B_{21} & A_{21} B_{12} + A_{22} B_{22}\\
\end{bmatrix}$$

Если $A_{11}, B_{11}$ и их произведение помещаются в кэш, то мы загружаем их в память только раз

# CPU vs GPU
---
<center><img src="pictures/bus_vs_plane.jpg" title="cpu_vs_gpu" width=500/></center>

# Отличия архитектур
---
<center><img src="pictures/architectures.png" width="500" align="center"/></center>
* ALU - Arithmetic logic unit

# Потребление энергии
---
<center><img src="https://ichef.bbci.co.uk/images/ic/720x405/p07p4w15.jpg" title="greta" width=200/></center>
<center><img src="pictures/energy.png" width="500" align="center"/><center/>

# Оптимизация задержки vs. Пропускная способность
---
<center><img src="pictures/threads.png" width="700" align="center"/><center/>

Устройство памяти GPU
---
<center><img src="pictures/gpu_memory.png" width="700" align="center"/><center/>

* Чтение данных в GPU крайне медленное, так как $L1$-кэш очень маленький. Вычисления в $1000$ раз быстрее, чем чтение данных.
<br>  
$\implies$ главным ботлнеком является чтение данных. Но как же тогда GPU ускоряет вычисления?

Произведение матриц на GPU
---
<center><img src="pictures/gpu_matrix1.png" width="600" align="center"/><center/>

**Элементарный шаг:**
* Чтение двух значений из матриц $M$ и $N$
* Умножение значений и сложение с предыдущим значением $P$  

**Вычислительная интенсивность:**
* 2 операции на 2 чтения из глобальной памяти  

**Если пропускная способность памяти - 300 GB/s:**
* Tesla K40L 4.29 TFLOPS (float) - **всего лишь 2% от пиковой производительности**

# Неэффективный доступ к элементам
---
<center><img src="pictures/gpu_threads1.png" width="600" align="center"/><center/>

# Блочный доступ к памяти (Tiling)
---
<center><img src="pictures/gpu_threads2.png" width="600" align="center"/><center/>

# Блочная схема умножения матриц
---
<center><img src="pictures/gpu_matrix2.png" width="600" align="center"/><center/>

# GPU for Deep Learning
---
https://devblogs.nvidia.com/optimizing-gpu-performance-tensor-cores/  
* Матрица может не делиться нацело между блоками (*tile quantization*)
* Потоки могут не делиться нацело между мультипроцессорами (*wave quantization*)

# Tile quantization
---
<a><img src="https://docs.nvidia.com/deeplearning/sdk/dl-performance-guide/graphics/tile-quant.png" title="tile" width=800/></a>
Tile quantization effect on (a) achieved flops throughput and (b) elapsed time, alongside (c) the number of tiles created. Measured with a function that forces the use of 256x128 tiles over the MxN output matrix. In practice, cuBLAS would select narrower tiles (for example, 64-wide) to reduce the quantization effect. Tesla V100-SXM3-32GB GPU, cuBLAS v10.1.

# Wave quantization
---
<a><img src="https://docs.nvidia.com/deeplearning/sdk/dl-performance-guide/graphics/wave-quant-effects.png" title="tile" width=800/></a>
The effects of wave quantization in terms of (a) achieved flops throughput and (b) elapsed time, as well as (c) the number of tiles created. Measured with a function that uses 256x128 tiles over the MxN output matrix. Note that the quantization effect occurs when the number of tiles passes a multiple of 80. Tesla V100-SXM3-32GB GPU, cuBLAS v10.1.

# Как быть?
---
У видеокарт различаются: 
* размер L1 и L2 кэша
* число ядер
* число потоков
<br>  
Оптимальные схемы вычислений могут быть разными на разных видеокартах.
<br>  
Вручную задавать гиперпараметры для разных видеокарт? К счастью, этого делать не нужно!

# Есть TensorRT!
---
* TensorRT ускоряет инференс нейронной сети
<a><img src="https://docs.nvidia.com/deeplearning/sdk/tensorrt-developer-guide/graphics/whatistrt2.png" title="tensorrt" width=800/></a>
https://docs.nvidia.com/deeplearning/sdk/tensorrt-best-practices/index.html
https://towardsdatascience.com/optimize-nvidia-gpu-performance-for-efficient-model-inference-f3e9874e9fdc

**Пример:**
<a><img src="https://devblogs.nvidia.com/wp-content/uploads/2018/06/perf_correct-1024x576.png" title="tensorrt_resnet" width=900/></a>

<center><b>Спасибо за внимание!</b><center/>